In [ ]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 73.6 MB/s eta 0:00:00


# Worksheet 3: Latin Square Problem in Z3

This week, you will use Z3 to solve the Latin square problem &ndash; a simplified version of Sudoku.

Latin square is a square matrix $M_{ij}$ of size $n \times n$ filled with elements $\{ 1, 2, \ldots, n \}$ such that each of them occurs exactly once in each row and column of $M_{ij}$.

A couple of examples of Latin squares of size $3 \times 3$:

$$
\begin{array}{ccc}
3 & 1 & 2\\
2 & 3 & 1\\
1 & 2 & 3\\
\end{array}
$$
and
$$
\begin{array}{ccc}
1 & 3 & 2\\
3 & 2 & 1\\
2 & 1 & 3\\
\end{array}
$$


You can also check the Wikipedia page: https://en.wikipedia.org/wiki/Latin_square

The associated problem is as follows.
You are given only some of the values in $M_{ij}$, and you are asked to fill in the rest of the values to satisfy the Latin square rules, or prove that no such values exist.

Examples of the Latin Square Problem instances.

* Satisfiable:
$$
\begin{array}{ccc}
? & 2 & ?\\
1 & 3 & ?\\
? & ? & 3\\
\end{array}
$$
* Unsatisfiable:
$$
\begin{array}{ccc}
? & ? & 1\\
2 & ? & ?\\
? & ? & 3\\
\end{array}
$$

<br/><br/>

First, let us define the Latin Square formally, with all the notations needed to describe the input data.

> _Definition_. Let $n$ be a positive integer.
Let $V$ be a set of tuples $(i, j, t)$ of integers, $1 \le i, j, t \le n$.
Then the Latin Square Problem is to find a matrix $M$ of size $n \times n$ such that each element of the matrix is an integer between 1 and $n$, there are no repeating elements in any row or column of the matrix, and $M_{i,j} = t$ for every $(i, j, t) \in V$.

## Latin Square Problem formulation

Now we can _formulate_ the problem in FOL.  A _formulation_ of a problem is a set of formulas that describe the problem for an arbitrary input (in our case, arbitrary values of $n$ and $V$).
In order to compose a problem formulation, we need to decide on the *solution representation*, i.e., on the predicates and functions that will describe the solution.  The Latin Square Problem seeks a combination of values in matrix $M$.  For the first few exercises, we will use FOL function $\text{tile}(x, y)$ to represent $M_{y, x}$.  Having a solution representation, it is easy to formulate the Latin square rules:
$$
\begin{split}
& \forall x . \forall y_1 . \forall y_2 . (y_1 \neq y_2) \rightarrow (\text{tile}(x, y_1) \neq \text{tile}(x, y_2)), \qquad & (1) \\
& \forall y . \forall x_1 . \forall x_2 . (x_1 \neq x_2) \rightarrow (\text{tile}(x_1, y) \neq \text{tile}(x_2, y)), \qquad & (2) \\
& \text{tile}(x, y) = t \qquad \forall (x, y, t) \in V, \qquad & (3) \\
& \mathcal{D} = \{ 1, 2, \ldots, n \} . \qquad & (4)
\end{split}
$$

Observe that equations (1)&ndash;(3) accurately describe our knowledge in FOL in the assumption that $n$ is the size of the domain of discourse, whereas equation (4) restricts the domain of discorse.  Strictly speaking, (4) is not even an FOL formula, however, without (4), the domain of discourse would be unrestricted and the formulation would not accurately reflect our problem.

How can we impose (4) in Z3?  There exist two simple approaches:
1. Unlike FOL, Z3 assigns a type to each variable, parameter and function.  We can define a new datatype that has exactly $n$ values, and use it for all our functions and variables.  You already have experience of defining such a datatype (recall the Blocks exercise in Worksheet&nbsp;2).

1. We can also use integers for all our variables and functions.  However, if we use integers, the domain of discourse is automatically infinitely large.  Thus, we have to add restrictions to our rules and solution representation.  Specifically, we have to state that the rules only apply when the variable values are between 1 and $n$, and that $1 \le \text{tile}(x, y) \le n$ for all $x$ and $y$.  For example, we can update (1) with
 \begin{multline*}
 \forall x . \forall y_1 . \forall y_2 . (x \ge 1) \land (x \le 3) \\
 \land (y_1 \ge 1) \land (y_1 \le 3) \land (y_2 \ge 1) \land (y_2 \le 3) \\
 \land (y_1 \neq y_2) \rightarrow \text{tile}(x, y_1) \neq \text{tile}(x, y_2)
 \end{multline*}

Neither of the two approaches is a pure FOL solution (why not?); we slightly bend the rules here to conveniently specify the size of the Latin Square.
You will see how to encode the same knowledge using pure FOL in the second part of the worksheet.


## Latin Square Problem Solver

Having a formulation, we can develop a Z3-based solver for the Latin Square Problem:
1. Given $n$ and $V$, encode the formulation (1)&ndash;(4) in Z3.

1. Run the solver.

1. If the program is unsatisfiable, print "No solution exists" and exit.

1. Otherwise, print a matrix of size $n \times n$ such that element in row $i$ and column $j$ is the value of $\text{tile}(i, j)$ in the interpretation found by Z3.

Note that we do not describe the algorithm to solve the Latin square problem; we use the declarative approach where we only describe the rules of the problem but leave the solution process to the reasoning software.  Much of this module will be dedicated to producing such knowledge bases and using them with off-the-shelf solvers (Z3 and later OR-Tools).


**Exercise 1.**  Your task is to implement a solver (in Python) using the first workaround, i.e. creating a new datatype to restrict the domain of discourse.

Hints:
1. To begin with, assume $n = 3$.  Generalise it for an arbitrary $n$ later.
1. The model will tell you the matrix values, however they will be hard to read.  Instead, you can use the `eval` function within your Python code, e.g.
```
s.model().eval(myfunction(1, 1))
```
will print out the value of function `myfunction` with parameter values 1 and 1.
1. To learn how to create a new Z3 datatype in Python, study the following example.



In [ ]:
# Example of creating a user-defined Z3 datatype in Python

from z3 import *

s = Solver()

myDatatype = z3.Datatype('MyDatatype')
myDatatype.declare('A')
myDatatype.declare('B')
myDatatype.declare('C')
myDatatype = myDatatype.create()



x = z3.Const('x', myDatatype)
y = z3.Const('y', myDatatype)
s.add(x != y)
s.add(x == myDatatype.B)

# Equivalent to s.add(z3.Or(y == myDatatype.B, y == myDatatype.C)).
# Equivalent to s.add(z3.Or(y == myDatatype.constructor(1)(), y == myDatatype.constructor(2)())).
s.add(z3.Or(*[y == myDatatype.constructor(i)() for i in range(1, 3)]))

if s.check() == unsat:
    print("unsat")
else:
    print(f'x = {s.model().eval(x)}')
    print(f'y = {s.model().eval(y)}')


x = B
y = C


In [59]:
# Solution to exercise 1
from z3 import *
s = Solver()
n = 10

coord = z3.Datatype('Coord')
for i in range (n):
  coord.declare(f'{i}')

coord = coord.create()

x = z3.Const('x', coord)
x1 = z3.Const('x1', coord)
x2 = z3.Const('x2', coord)
y = z3.Const('y', coord)
y1 = z3.Const('y1', coord)
y2 = z3.Const('y2', coord)
t = z3.Const('t', coord)

tile = z3.Function('tile', coord, coord, coord)
s.add(
    z3.ForAll ([x1, x2, y],
                z3.Implies(
                   (z3.Not (x1 == x2) ),
                    (z3.Not ((tile(x1,y)) == (tile (x2, y))))
                )
               )
)
s.add(
    z3.ForAll ([y1, y2, x],
                z3.Implies(
                   (z3.Not (y1 == y2) ),
                    (z3.Not ((tile(x,y1)) == (tile (x, y2))))
                )
               )
)

s.add(z3.And(*[tile(coord.constructor(i)(),coord.constructor(j)()) ==  coord.constructor(k)() for i,j,k in range(n,n,n)]))



if s.check() == unsat:
    print("unsat")
else:
  for i in range (n):
    print("")
    for j in range (n):
      print(s.model().eval(tile(coord.constructor(i)(),coord.constructor(j)())),end='')





5691427380
0918236475
2845961037
3729504618
6503798124
4280673951
7456189203
8137042569
1364850792
9072315846

Test your solver on satisfiable and unsatisfiable instances.

**Exercise 2.**  Now produce an alternative solution that would use the second work-around, i.e. would use integers while restricting the domain of application of the for-all quantifiers.

Question: how does Z3 define the `tile` function for column and row indices outside the $[1,3]$ range in its model?

In [65]:
# Solution to exercise 2
from z3 import *
import itertools
s = Solver()
x = z3.Int('x')
y = z3.Int('y')
x1 = z3.Int('x1')
y1 = z3.Int('y1')
x2 = z3.Int('x2')
y2 = z3.Int('y2')
t = z3.Int('t')
n = 3
tile = z3.Function('tile', IntSort(), IntSort(),IntSort())

s.add(
    z3.ForAll([x,y1,y2], z3.Implies (
              (z3.And(
                  (z3.And((
                    z3.And((y2 >= 1), (y2 <= n))), (z3.And (
                        (z3.And((x>= 1), (x <= n))), (z3.And((y1>= 1), (y1 <= n)))
                      ))
                  )), (z3.Not (y2 == y1))
              )), (z3.Not(tile(x,y1) == tile(x,y2))))
              )
    )

s.add(
    z3.ForAll([y,x1,x2], z3.Implies (
              (z3.And(
                  (z3.And((
                    z3.And((x2 >= 1), (x2 <= n))), (z3.And (
                        (z3.And((y >= 1), (y <= n))), (z3.And((x1 >= 1), (x1 <= n)))
                      ))
                  )), (z3.Not (x2 == x1))
              )), (z3.Not(tile(x1,y) == tile(x2,y))))
              )
    )

s.add(
    z3.ForAll([x,y],
    # z3.And(
        z3.Implies (
              (z3.And
                ((z3.And((y >= 1), (y <= n))), (z3.And((x>=1), (x <= n))))
               ), (z3.And((tile(x,y)<= n),(tile (x,y) >= 1)))

        ),

    )
)
# )


if s.check() == unsat:
    print("unsat")
else:
    for i in range (1,n+1):
      print("")
      for j in range (1,n+1):
        print(s.model().eval(tile(i,j)), end = '')




231
123
312

**Exercise 3.**  Which of the two solvers is faster?  To answer this question, increase $n$.  Also play with various fixed values in the matrix; what makes the instances computationally hard (i.e. significantly slows does the Z3 solver)?

Your answer to exercise 3:

The first one is faster. It contains less definations and may spend less time when finding a solution.

## Explicit enumeration of constraints

In Exercise 3, you observed that even relatively small changes to the formulation may have considerable effect on the performance of Z3.  It is hard to pinpoint what exactly makes Z3 slower or faster; the solver is too complex to identify the bottleneck without thorough research.  However, it is often relatively easy to restructure the knowledge base and test if the new structure leads to better solution times.  Practitioners would usually try several knowledge base structures and choose the most effective one by trial and error.

In this exercise you will use a different formulation of the Latin Square Problem to build a new solver.

### New formulation of the Latin Square Problem

The the Latin Square Problem can be formulated as follows.

Let $N = \{ 1, 2, \ldots, n \}$ &ndash; we introduce this set for convenience.  Our solution representation here will be a set of constants (functions of arity zero) $m_{i,j}$ for $i, j \in N$.  Specifically, $m_{i,j}$ will be the value of tile at coordinates $(i, j)$.  Then the Latin Square Problem can be formulated as follows.
$$
\begin{split}
& m_{i,j} \neq m_{k,j} \qquad & \forall i < k \in N,\ \forall j \in N, \qquad & (5)\\
& m_{i,j} \neq m_{i,k} & \forall i \in N,\ \forall j < k \in N, \qquad & (6)\\
& m_{i,j} = t & \forall (i, j, t) \in V, \qquad & (7)\\
& \mathcal{D} = N & & (8)
\end{split}
$$

Equations (5) and (6) define the Latin square rules: no two elements in a row/column can be the same.  The known values in the matrix are fixed by (7).  Equation (8) restricts the domain of discourse as otherwise the tile values would be unbound.

### Discussion of the new formulation

Unlike the formulation (1)&ndash;(4), formulation (5)&ndash;(8) uses individual constants for each tile.  This increases the number of sentences in our knowledge base, however it is still of finite size.  Indeed, each index on the right of this formulation can only take a finite number of values.  Thus, we can explicitly enumerate all possible combinations of indices instead of using quantifiers.  For example, for $n = 3$, (5) expands as
$$
\begin{split}
& m_{1,1} \neq m_{2,1}\\
& m_{1,1} \neq m_{3,1}\\
& m_{2,1} \neq m_{3,1}\\
& m_{1,2} \neq m_{2,2}\\
& m_{1,2} \neq m_{3,2}\\
& m_{2,2} \neq m_{3,2}\\
& m_{1,3} \neq m_{2,3}\\
& m_{1,3} \neq m_{3,3}\\
& m_{2,3} \neq m_{3,3}
\end{split}
$$

The expanded formulation consists of many sentences; $O(n^3)$ sentences to be precise.
However, this does not make it inefficient; packages such as Z3 can handle large formulations, and languages such as Python allow creating and feeding such formulations to Z3.

**Exercise 4.**  Study the discussion in the Explicit Enumeration of Constraints section and the new formulation of the Latin Square problem.
Pay attention to how the formulation is described:
* Auxiliary notation $N$ is defined above the formulas;
* Each group of rules is defined in a separate line;
* Groups of rules can be parametrised, with the domain of each parameter being defined on the right; for example, in (5), we specify that the rule is defined for every combination of $i \in N$, $j \in N$ and $k \in N$ such that $i < k$;
* Formula indices are given in brackets, for later references (include the index even if you never reference this specific formula);
* Each line of the formulation is explained below the formal formulation;
* To refer to the formulation, we give the range of formula indices, e.g., "formulation&nbsp;(5)&ndash;(8)".

In your coursework projects, you will need to define formulations in this style.

**Exercise 5.**  Write a Python program that encodes the formulation (5)&ndash;(8) and solves it using Z3.  (Do not use SMT2; your solution would not be generic enough.)

You may use your own data type for the elements of $M$ or you may use integers, in which case you would need the restrict their values.

Separate the problem input from the solution method.  Below is a template that will help you in that.

In [70]:
# Solution to exercise 5

def solve_latin_square(n, V):
    solver = z3.Solver()
    # Your code here

    # i = z3.Int ('i')
    # j = z3.Int ('j')
    # k = z3.Int ('k')
    # t = z3.Int ('t')
    tile = z3.Function('tile', IntSort(), IntSort(),IntSort())
    for i in range (1,n + 1):
      for j in range (1, n+1):
        for k in range (j+1, n+1):
          solver.add(z3.Not(tile(i, j) == tile(i,k)))

    for j in range (1,n + 1):
      for i in range (1, n+1):
        for k in range (i+1, n+1):
          solver.add(z3.Not(tile(i, j) == tile(k,j)))

    for i in range(0, len(V)):
      t = V[i]
      solver.add(tile(t[0], t[1]) == t[2])

    for i in range(1, n +1):
      for j in range (1, n+1):
        solver.add(z3.And((tile(i,j) <= n),(tile(i,j) >= 1)))

    if solver.check() == unsat:
      print("unsat")
    else:
      for i in range (1,n+1):
        print("")
        for j in range (1,n+1):
          print(solver.model().eval(tile(i,j)), end = '')



solve_latin_square(10, [(1, 1, 2), (2, 2, 3)])


26510741389
10376915428
71042358916
97651832104
81271069543
58394271061
45816327910
32948106157
19108274635
64135910872

**Exercise 6.**  Compare the solution times for all the formulations that you used.  Which approach is faster?  Use identical instance data to compare the methods.


Your answer to exercise 6:

???